In [1]:
%matplotlib inline
from sklearn import datasets, linear_model
import matplotlib.pyplot as plt
import re
import seaborn as sns

import pandas as pd
import numpy as np
import datetime
import pandas as pd
import timeit

from sklearn.neural_network import MLPRegressor 
from sklearn.model_selection import train_test_split

In [2]:
df_airquality = pd.DataFrame.from_csv('data/airquality.csv')
df_district = pd.DataFrame.from_csv('data/district.csv')
df_meteorology = pd.DataFrame.from_csv('data/meteorology.csv')
df_station = pd.DataFrame.from_csv('data/station.csv')
df_weatherforecast = pd.DataFrame.from_csv('data/weatherforecast.csv')

In [3]:
final_df = df_airquality.join(df_station)
final_df = final_df.reset_index()
final_df['station_id'] = final_df['\ufeffstation_id']
final_df = final_df.drop('\ufeffstation_id',1)
final_df = final_df.join(df_district, on=['district_id'],rsuffix='_district')

In [4]:
df_city = pd.DataFrame.from_csv('data/city.csv')
df_city = df_city.reset_index()

df_city['city_id'] = df_city['\ufeffcity_id']
df_city['latitude_city']= df_city['longitude']
df_city['longitude_city']= df_city['longitude']

df_city = df_city.drop('latitude',1)
df_city = df_city.drop('longitude',1)

df_city = df_city.drop('\ufeffcity_id',1)
df_city['city_id'] = df_city.city_id.apply(lambda x : int(x))
final_df = pd.merge(final_df, right=df_city,on=['city_id'],how='left')

In [5]:
final_df = final_df.drop('name_chinese_x',1)
final_df = final_df.drop('name_chinese_district',1)
final_df = final_df.drop('name_chinese_y',1)
final_df = final_df.drop('name_english_y',1)

In [6]:
df_meteorology = df_meteorology.reset_index()
df_meteorology['id'] = df_meteorology['\ufeffid']
df_meteorology = df_meteorology.drop('\ufeffid',1)

In [7]:
df_meteorology['district_id'] = df_meteorology['id']
df_meteorology = df_meteorology.drop('id',1)
final_df = pd.merge(final_df, right=df_meteorology,on=['district_id','time'],how='left')

In [8]:
final_df['latitude_city']  = final_df['latitude_city'].apply(lambda x : str(x))
final_df['longitude_city']  = final_df['longitude_city'].apply(lambda x : str(x))

final_df['latitude']  = final_df['latitude'].apply(lambda x : str(x))
final_df['longitude']  = final_df['longitude'].apply(lambda x : str(x))

final_df['lat-lon-city'] = final_df['latitude_city'] + ' ' + final_df['longitude_city']
final_df['lat-lon-district'] = final_df['latitude'] + ' ' + final_df['longitude']

In [9]:
final_df = final_df.drop('latitude_city',1)
final_df = final_df.drop('longitude_city',1)

final_df = final_df.drop('latitude',1)
final_df = final_df.drop('longitude',1)

final_df['time'] = final_df.time.apply(lambda x : datetime.datetime.strptime(str(x),'%Y-%m-%d %H:%M:%S'))

In [10]:
final_df['year'] = final_df['time'].apply(lambda x : x.year)
final_df['month'] = final_df['time'].apply(lambda x : x.month)
final_df['day'] = final_df['time'].apply(lambda x : x.day)
final_df['hour'] = final_df['time'].apply(lambda x : x.hour)
final_df['minute'] = final_df['time'].apply(lambda x : x.minute)
final_df['seconds'] = final_df['time'].apply(lambda x : x.second)

final_df = final_df.drop('time',1)

In [11]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2891393 entries, 0 to 2891392
Data columns (total 26 columns):
PM25_Concentration       float64
PM10_Concentration       float64
NO2_Concentration        float64
CO_Concentration         float64
O3_Concentration         float64
SO2_Concentration        float64
name_english_x           object
district_id              int64
station_id               int64
name_english_district    object
city_id                  int64
cluster_id               int64
weather                  float64
temperature              float64
pressure                 float64
humidity                 float64
wind_speed               float64
wind_direction           float64
lat-lon-city             object
lat-lon-district         object
year                     int64
month                    int64
day                      int64
hour                     int64
minute                   int64
seconds                  int64
dtypes: float64(12), int64(10), object(4)
memory usag

In [12]:
# Save it to a temporary data frame 
# final_df.to_csv('data/final_dataframe.csv')

In [13]:
# final_df = pd.DataFrame.from_csv('data/final_dataframe.csv')

In [14]:
final_df.columns

Index(['PM25_Concentration', 'PM10_Concentration', 'NO2_Concentration',
       'CO_Concentration', 'O3_Concentration', 'SO2_Concentration',
       'name_english_x', 'district_id', 'station_id', 'name_english_district',
       'city_id', 'cluster_id', 'weather', 'temperature', 'pressure',
       'humidity', 'wind_speed', 'wind_direction', 'lat-lon-city',
       'lat-lon-district', 'year', 'month', 'day', 'hour', 'minute',
       'seconds'],
      dtype='object')

In [15]:
final_df.head()

PM25_Concentration  PM10_Concentration  NO2_Concentration  \
0               138.0               159.4               56.3   
1               124.0               163.9               38.7   
2               127.0               148.4               55.6   
3               129.0               145.6               65.7   
4               119.0               119.3               66.9   

   CO_Concentration  O3_Concentration  SO2_Concentration     name_english_x  \
0               0.9              50.8               17.2  HaiDianBeiBuXinQu   
1               0.9              51.1               17.9  HaiDianBeiBuXinQu   
2               1.0              27.2               16.6  HaiDianBeiBuXinQu   
3               1.0               9.7               16.7  HaiDianBeiBuXinQu   
4               1.0               2.0               16.5  HaiDianBeiBuXinQu   

   district_id  station_id name_english_district   ...    wind_speed  \
0          101        1001             HaiDianQu   ...          7.92   
1          101        1001             HaiDianQu   ...          7.56   
2          101        1001             HaiDianQu   ...          5.76   
3          101        1001             HaiDianQu   ...          6.12   
4          101        1001             HaiDianQu   ...          4.68   

   wind_direction           lat-lon-city               lat-lon-district  year  \
0            13.0  116.407394 116.407394  40.090678999999994 116.173553  2014   
1            13.0  116.407394 116.407394  40.090678999999994 116.173553  2014   
2            13.0  116.407394 116.407394  40.090678999999994 116.173553  2014   
3            13.0  116.407394 116.407394  40.090678999999994 116.173553  2014   
4             1.0  116.407394 116.407394  40.090678999999994 116.173553  2014   

   month  day  hour minute seconds  
0      5    1     0      0       0  
1      5    1     1      0       0  
2      5    1     2      0       0  
3      5    1     3      0       0  
4      5    1     4      0       0  

[5 rows x 26 columns]

In [16]:
train_x_numerical = [ 'year', 'month', 'day', 'hour' , 'minute', 'seconds', 'district_id', 'city_id']
train_x_string = ['name_english_x', 'lat-lon-city', 'lat-lon-district',  'name_english_district', ]           
train_x_weather = ['weather', 'temperature', 'pressure', 'humidity','wind_speed', 'wind_direction']

train_columns = []
train_columns.extend(train_x_numerical)
train_columns.extend(train_x_weather)

train_y_pollution = ['PM25_Concentration', 'PM10_Concentration', 'NO2_Concentration', 'CO_Concentration', 'O3_Concentration', 'SO2_Concentration']


In [17]:
df_null = final_df[train_columns][final_df[train_columns].isnull().any(axis=1)]
df_null.head(10)

year  month  day  hour  minute  seconds  district_id  city_id  weather  \
142   2014      5    8    23       0        0          101        1      NaN   
192   2014      5   11     1       0        0          101        1      5.0   
601   2014      5   28     3       0        0          101        1      0.0   
812   2014      6    6    22       0        0          101        1      0.0   
813   2014      6    6    23       0        0          101        1      0.0   
940   2014      6   12     6       0        0          101        1      0.0   
962   2014      6   13     9       0        0          101        1      NaN   
975   2014      6   14     5       0        0          101        1      2.0   
1054  2014      6   18     3       0        0          101        1      2.0   
1170  2014      6   22    23       0        0          101        1      0.0   

      temperature  pressure  humidity  wind_speed  wind_direction  
142          17.0    1010.0       0.0        5.76            13.0  
192           NaN       NaN       NaN         NaN            14.0  
601          18.0     998.0       0.0         NaN            23.0  
812           NaN       NaN       NaN         NaN             4.0  
813           NaN       NaN       NaN         NaN             4.0  
940          20.0    1004.0       0.0         NaN             4.0  
962          28.0    1004.0       0.0       11.52             9.0  
975          19.0    1004.0       0.0         NaN             4.0  
1054         19.0    1000.0       0.0         NaN             9.0  
1170         19.0    1006.0       0.0         NaN             9.0

### Trying simple logistic regression for district predictons 

1. Type 1 fill NaN with 0 to see how the things are performing 
2. Remove rows with NaN to see what's the impact 

In [18]:
df = final_df.fillna(0.0)
train_df, test_df = train_test_split(df, test_size = 0.2)
print(len(train_df) , len(test_df))

# Create linear regression object
regr = linear_model.LinearRegression()

# Train the model using the training sets
regr.fit(train_df[train_columns], train_df[train_y_pollution])

# The coefficients
print('\nCoefficients: \n', regr.coef_)
# The mean squared error
print("\n\nMean squared error for " )
print(np.mean((regr.predict(test_df[train_columns]) - test_df[train_y_pollution]) ** 2))
# Explained variance score: 1 is perfect prediction
print('\nVariance score: %.2f' % regr.score(test_df[train_columns], test_df[train_y_pollution]))

2313114 578279

Coefficients: 
 [[  2.05779972e+01   2.67270328e+00   4.35646848e-01  -5.98522880e-02
    1.75970349e-13  -4.89386309e-13   1.31129416e+00  -1.31115621e+02
    7.43905098e+00  -1.38934958e+00   1.04182082e-02   2.04732378e-01
   -1.65157234e+00   4.67988399e-01]
 [  4.03966348e+01   4.25817170e+00   1.08181157e+00   2.05631154e-01
    3.45945494e-13  -8.81961171e-13   2.35485589e+00  -2.35374206e+02
    7.03582171e+00  -1.88240478e+00   1.97546622e-02  -5.80280202e-02
   -1.41071594e+00   8.98829736e-01]
 [  1.48436806e+01   1.68720900e+00   1.36391010e-01   1.21168342e-01
    4.79616347e-14  -1.21902488e-13   2.51942240e-01  -2.52143551e+01
    1.75118578e+00  -3.65168165e-01   8.67108054e-03   2.72088854e-02
   -1.12149524e+00   6.32350761e-02]
 [  3.65915305e-01   4.41737784e-02   3.92115646e-03  -2.79805560e-03
    1.47798440e-15  -3.72618603e-15   8.57472545e-03  -8.55724550e-01
    8.49144721e-02  -3.14819492e-02   2.62976018e-04   4.91799973e-03
   -3.55608832e-0

In [19]:
# Removing entries with NaN values in the regression model
# Helped to impove scores
df = final_df[~final_df.isnull().any(axis=1)]
train_df, test_df = train_test_split(df, test_size = 0.2)
print(len(train_df) , len(test_df))

# Create linear regression object
regr = linear_model.LinearRegression()

# Train the model using the training sets
regr.fit(train_df[train_columns], train_df[train_y_pollution])

# The coefficients
print('\nCoefficients: \n', regr.coef_)
# The mean squared error
print("\n\nMean squared error for " )
print(np.mean((regr.predict(test_df[train_columns]) - test_df[train_y_pollution]) ** 2))
# Explained variance score: 1 is perfect prediction
print('\nVariance score: %.2f' % regr.score(test_df[train_columns], test_df[train_y_pollution]))

620164 155042

Coefficients: 
 [[ -3.13376554e+01  -2.58500527e+00   6.63202904e-01   4.02987927e-01
   -1.40648604e-12  -1.07358566e-13   1.91095038e+00  -1.90896915e+02
    7.25080858e+00  -1.94913598e+00   4.65724866e-01   2.55421956e-01
   -1.79113711e+00   6.66858133e-01]
 [ -3.10822846e+01  -2.72910221e+00   1.62674139e+00   4.92650674e-01
   -2.40463205e-12  -1.56430424e-13   3.31310281e+00  -3.30983970e+02
    9.35646641e+00  -2.32860740e+00   5.76529249e-01  -1.14895001e-01
   -1.50255864e+00   1.04703258e+00]
 [ -1.90925230e+01  -1.82122876e+00   1.81028320e-01   2.71911608e-01
   -2.25541807e-13  -6.53366250e-14   2.97598360e-01  -2.96817591e+01
    1.49100973e+00  -7.95645616e-01   2.49818695e-01   5.79366852e-02
   -1.28824602e+00   1.16232582e-01]
 [ -7.03460357e-01  -6.70162560e-02   7.64841136e-03   2.09964272e-03
   -1.17362717e-14  -2.73880311e-15   1.61688235e-02  -1.61245038e+00
    8.89521322e-02  -4.30711508e-02   8.41557338e-03   3.63080014e-03
   -3.69211700e-02

#### Using averages to estimate values and rerunning the regressions on the dataframe

In [24]:
merged_df = final_df.groupby(train_x_numerical).mean().reset_index()
print ("With remove NaN's ", len(final_df[~final_df.isnull().any(axis=1)]))
print ("Without removed Nan's " , len(final_df))
print ("Length of DF with merged ", len(merged_df[~merged_df.isnull().any(axis=1)]))

With remove NaN's  775206
Without removed Nan's  2891393
Length of DF with merged  615432


In [21]:
# Removing entries with NaN values in the regression model
# Helped to impove scores
df = merged_df[~merged_df.isnull().any(axis=1)].fillna(0.0)
train_df, test_df = train_test_split(df, test_size = 0.2)
print(len(train_df) , len(test_df))

# Create linear regression object
regr = linear_model.LinearRegression()

# Train the model using the training sets
regr.fit(train_df[train_columns], train_df[train_y_pollution])

# The coefficients
print('\nCoefficients: \n', regr.coef_)
# The mean squared error
print("\n\nMean squared error for " )
print(np.mean((regr.predict(test_df[train_columns]) - test_df[train_y_pollution]) ** 2))
# Explained variance score: 1 is perfect prediction
print('\nVariance score: %.2f' % regr.score(test_df[train_columns], test_df[train_y_pollution]))

492345 123087

Coefficients: 
 [[ -2.34139378e+01  -1.85184286e+00   6.53963899e-01   4.80020942e-01
   -4.82613949e-13  -1.59339208e-12   1.96733907e+00  -1.96554040e+02
    7.50571573e+00  -1.87752856e+00   4.80545760e-01   3.45271967e-01
   -1.92880709e+00   7.03824757e-01]
 [ -1.33672475e+01  -9.86669253e-01   1.68104048e+00   5.50891830e-01
   -8.01358979e-13  -2.77400325e-12   3.38263609e+00  -3.37952616e+02
    9.69765753e+00  -2.13282699e+00   6.01215580e-01   5.69409866e-03
   -1.63947875e+00   1.09172362e+00]
 [ -1.44648419e+01  -1.33260111e+00   1.64929650e-01   2.92789278e-01
   -6.47329412e-14  -2.48842613e-13   3.28957862e-01  -3.28192155e+01
    1.51061451e+00  -7.47665232e-01   2.45050597e-01   7.67386056e-02
   -1.28764030e+00   9.54303609e-02]
 [ -6.71527036e-01  -6.54116439e-02   7.93132408e-03   1.97858819e-03
   -2.99955373e-15  -1.11260827e-14   1.49245596e-02  -1.48823744e+00
    9.18072224e-02  -4.40008752e-02   8.51965945e-03   3.72326463e-03
   -3.92311245e-02

### Using a more complicated neural network based approach

Multi-layer Perceptron is sensitive to feature scaling, so it is highly recommended to scale your data. For example, scale each attribute on the input vector X to `[0, 1]` or `[-1, +1]`, or standardize it to have mean `0` and variance `1`. Note that you must apply the same scaling to the test set for meaningful results. You can use StandardScaler for standardization.


In [ ]:
merged_df = final_df.groupby(train_x_numerical).mean().reset_index()
mask = ~merged_df.isnull().any(axis=1)
df = merged_df[mask]
train_df, test_df = train_test_split(df, test_size = 0.2)

X_train = train_df[train_columns]
Y_train = train_df[train_y_pollution]

X_test = train_df[train_columns]
Y_test = train_df[train_y_pollution]

scaler = StandardScaler()  
scaler.fit(X_train)  
X_train = scaler.transform(X_train)  
X_test = scaler.transform(X_test)  

regr = MLPRegressor()


print('Train / Test ', len(train_df) , len(test_df))
print('Training neural Network model might take some time')
start = timeit.default_timer()
regr.fit( X_train , Y_train)
stop = timeit.default_timer()
train_time = int(stop - start)
print('Training finshed in ',train_time , 'seconds')
print("\n\nMean squared error for ")
print(np.mean((regr.predict(X_test) - Y_test) ** 2))
print('\nVariance score: %.2f' % regr.score(X_test, Y_test))

492345 123087
492345 123087
Training neural Network model might take some time


We can see, using a simple neural network based classifier we can improve on the variance by a lot.

### Running for all the data without merging the data.

In [ ]:
train_df, test_df = train_test_split(final_df, test_size = 0.2)

print(len(train_df) , len(test_df))

X_train = train_df[train_columns]
Y_train = train_df[train_y_pollution]

X_test = train_df[train_columns]
Y_test = train_df[train_y_pollution]

scaler = StandardScaler()
scaler.fit(X_train)  
X_train = scaler.transform(X_train)  
X_test = scaler.transform(X_test)  

regr = MLPRegressor()

print(len(train_df) , len(test_df))
print('Training neural Network model might take some time')
start = timeit.default_timer()
regr.fit( X_train , Y_train)
stop = timeit.default_timer()
train_time = int(stop - start)
print('Training finshed in ',train_time , 'seconds')
print("\n\nMean squared error for ")
print(np.mean((regr.predict(X_test) - Y_test) ** 2))
print('\nVariance score: %.2f' % regr.score(X_test, Y_test))